In [1]:
import random

In [2]:
class Barco:
    def __init__(self, nombre, tamano):
        self.nombre = nombre    # Variable para definir el atributo nombre del barco. 
        self.tamano = tamano    # Variable para definir el atributo tamaño / longitud del barco.
        self.posicion = []      # Variable para definir el tributo de la posición de los barcos en el tablero.
        self.golpes = 0         # Variable para definir el atributo de los golpes que recibirá el barco contando desde cero.

    def recibir_golpe(self): # Metodo que incrementa el contador de golpes en el barco.
        self.golpes += 1

    def esta_hundido(self): # Metodo que cálcula si el barco ha sido hundido.
        return self.golpes >= self.tamano

In [3]:
class Tablero:
    def __init__(self):
        self.tablero = [['O' for _ in range(10)] for _ in range(10)]  # Tablero 10x10 inicializado con 'O' (agua)
        self.barcos = []  # Lista de barcos colocados en el tablero

    def colocar_barco(self, barco, fila, columna, orientacion):
        posiciones = []
        
        # Generar posiciones basadas en la orientación
        for i in range(barco.tamano):
            if orientacion == 'H':
                if columna + barco.tamano > 10:  # Definiendo que los barcos no salgan del limite horizontal del tablero.
                    return False
                posiciones.append((fila, columna + i))
            elif orientacion == 'V':
                if fila + barco.tamano > 10:  # Definiendo que los barcos no salgan del limite vertical del tablero.
                    return False
                posiciones.append((fila + i, columna))

        # Verificar que las posiciones estén libres
        for (f, c) in posiciones:
            if self.tablero[f][c] != 'O':
                return False  # Posición ocupada, no se puede colocar el barco

        # Colocar el barco en el tablero
        for (f, c) in posiciones:
            self.tablero[f][c] = 'B'  # Marcar la posición con 'B' de barco
        barco.posicion = posiciones
        self.barcos.append(barco)
        return True

    def recibir_ataque(self, fila, columna):
        if self.tablero[fila][columna] == 'B':
            self.tablero[fila][columna] = 'X'  # 'X' marca un golpe
            for barco in self.barcos:
                if (fila, columna) in barco.posicion:
                    barco.recibir_golpe()
                    if barco.esta_hundido():
                        print(f"¡El barco {barco.nombre} ha sido hundido!")
                    return True  # Golpe
        elif self.tablero[fila][columna] == 'O':
            self.tablero[fila][columna] = '-'  # '-' marca un fallo
        return False  # Fallo

    def mostrar_tablero(self):
        # Encabezado de columnas
        print("  " + " ".join(map(str, range(10))))
        for i, fila in enumerate(self.tablero):
            # Imprimir número de fila al principio de cada línea y luego los valores de la fila
            print(f"{i} " + " ".join(fila))

    def todos_barcos_hundidos(self):
        return all(barco.esta_hundido() for barco in self.barcos)

In [4]:
class Jugador:
    def __init__(self, nombre):
        self.nombre = nombre
        self.tablero = Tablero()

    def atacar(self, oponente, fila, columna):
        resultado = oponente.tablero.recibir_ataque(fila, columna)
        if resultado:
            print(f"{self.nombre} golpeó un barco en la coordenada ({fila}, {columna})!")
        else:
            print(f"{self.nombre} falló en la coordenada ({fila}, {columna}).")
        return resultado

In [5]:
class Juego:
    def __init__(self, jugador1, jugador2):
        self.jugador1 = jugador1
        self.jugador2 = jugador2

    def colocar_barcos_iniciales(self, jugador):
        barcos = [
            ("Portaavion", 5),
            ("Acorazado", 4),
            ("Crucero", 3),
            ("Submarino", 3),
            ("Destructor", 2)
            ]
        for nombre, tamano in barcos:
            colocado = False
            while not colocado:
                fila = random.randint(0, 9)
                columna = random.randint(0, 9)
                orientacion = random.choice(['H', 'V'])
                barco = Barco(nombre, tamano)
                colocado = jugador.tablero.colocar_barco(barco, fila, columna, orientacion)

    def iniciar_juego(self):
        print("¡Iniciando partida de Battleship!")
        self.colocar_barcos_iniciales(self.jugador1)
        self.colocar_barcos_iniciales(self.jugador2)
        
        turno_actual = self.jugador1
        oponente = self.jugador2
        
        while not (self.jugador1.tablero.todos_barcos_hundidos() or self.jugador2.tablero.todos_barcos_hundidos()):
            self.turno(turno_actual, oponente)
            turno_actual, oponente = oponente, turno_actual  # Cambiar turnos

        ganador = self.jugador1 if self.jugador2.tablero.todos_barcos_hundidos() else self.jugador2
        print(f"¡{ganador.nombre} ha ganado el juego!")

    def turno(self, jugador, oponente):
        print(f"\nTurno de {jugador.nombre}")
        oponente.tablero.mostrar_tablero()
        
        # Solicitar coordenadas de ataque al jugador
        fila = int(input("Ingresa la fila del (0-9): "))
        columna = int(input("Ingresa la columna del (0-9): "))
        
        # Verificar que no se ataque la misma posición más de una vez
        if oponente.tablero.tablero[fila][columna] in ['X', '-']:
            print("Ya atacaste esa posición. Elige otra coordenada.")
            return self.turno(jugador, oponente)
        
        jugador.atacar(oponente, fila, columna)

In [ ]:
jugador1 = Jugador("Jugador 1")
jugador2 = Jugador("Jugador 2")
juego = Juego(jugador1, jugador2)
juego.iniciar_juego()